In [ ]:
import os
import pandas as pd
import pickle
import numpy as np
import nibabel as nib

def fc_matrix(ts, cutoff=5):
    ts = ts[cutoff:]

    fc = np.corrcoef(ts, rowvar=False)

    return fc

atlas = "DesikanKilliany"
derivatives_dir = "/Users/brainsimulation/Desktop/derivatives_dk_T2uniform/derivatives"
os.makedirs(f"out/{atlas}", exist_ok=True)
demo_data = pd.read_csv("/Users/brainsimulation/Desktop/NMDARE_FC_Analysis/out/DesikanKilliany/demo_data.csv")
excluded = ['RSGK017', 'LENMDA001', 'LENMDA006', 'LENMDA172', 'LENMDA187', 'LENMDA191']
subids = [subid[4:] for subid in os.listdir(os.path.join(derivatives_dir, "xcp_d")) if os.path.exists(os.path.join(derivatives_dir, "xcp_d", f"{subid}", "func", f"{subid}_task-rest_space-fsLR_seg-{atlas}_den-91k_stat-pearsoncorrelation_boldmap.pconn.nii")) and subid[4:] not in excluded]

with open(f'/Users/brainsimulation/Desktop/NMDARE_FC_Analysis/out/{atlas}/to_delete_network.pkl', 'rb') as f:
    to_delete_network = pickle.load(f)

with open(f'/Users/brainsimulation/Desktop/NMDARE_FC_Analysis/out/{atlas}/network_labels.pkl', 'rb') as f:
    network_labels = pickle.load(f)

nmdare_fcs = []
hc_fcs = []

for subid in subids:

    xcp_func_folder = os.path.join(derivatives_dir, "xcp_d", f"sub-{subid}", "func")

    ts_fname = os.path.join(xcp_func_folder, f"sub-{subid}_task-rest_space-fsLR_seg-{atlas}_den-91k_stat-mean_timeseries.ptseries.nii")    
    ts = np.array(nib.load(ts_fname).dataobj)
    ts = np.delete(ts, to_delete_network, 1)

    fc = fc_matrix(ts)

    os.makedirs(f"sub-{subid}", exist_ok=True)
    np.savetxt(f"sub-{subid}/FC.csv", fc, delimiter=',')

    if demo_data.loc[demo_data["ID"] == subid].nmdare.item() == 1:
        nmdare_fcs.append(fc)
    elif demo_data.loc[demo_data["ID"] == subid].nmdare.item() == 0:
        hc_fcs.append(fc)

avg_nmdare_fc = np.mean(nmdare_fcs, axis=0)
avg_hc_fc = np.mean(hc_fcs, axis=0)

np.savetxt(f"avg_nmdare_fc.csv", avg_nmdare_fc, delimiter=',')
np.savetxt(f"avg_hc_fc.csv", avg_hc_fc, delimiter=',')

/var/folders/mn/q624n0lj15s26j52tyhh602h0000gn/T/ipykernel_85259/1013974118.py:37: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  ts = np.array(nib.load(ts_fname).dataobj)
